# Gathering all the data, then doing a clean for a master dataset.

In [1]:
# Import the needed libraries
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools


In [2]:
# Load CSV data sets into a dataframe

loc = "/mnt/c/Users/anton/Documents/Projects/Predicting_WaterlooHealthInspections/datasets/"

raw_dataset_0 = pd.read_csv(loc + 'Facilities_OpenData.csv')
raw_dataset_1 = pd.read_csv(loc + 'Infractions_OpenData.csv')
raw_dataset_2 = pd.read_csv(loc + 'Inspections_OpenData.csv')

In [3]:
# Parse data as time 

def parse_date1(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d').date()

# two different date formats present
def parse_date2(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y/%m/%d').date()


raw_dataset_1.InspectionDate = raw_dataset_1.InspectionDate.apply(parse_date1)
raw_dataset_2.INSPECTION_DATE = raw_dataset_2.INSPECTION_DATE.apply(parse_date2)


In [4]:
# Get relevant columns per raw dataset

facilities_col_req = ["FACILITYID", "BUSINESS_NAME", "ADDR", "CITY", "CATEGORY", "SUBCATEGORY"]
infractions_col_req = ["INSPECTION_ID", "INFRACTION_TYPE", "Result", "InspectionDate"] #certain fields ignored, no value or inability to parse in a useful manner
inspections_col_req = ["INSPECTION_ID", "FACILITYID", "INSPECTION_DATE", "REQUIRE_REINSPECTION", "CERTIFIED_FOOD_HANDLER", "INSPECTION_TYPE"]

# Create new dataframe with required columns

data_facilities = raw_dataset_0[facilities_col_req]
data_infractions = raw_dataset_1[infractions_col_req]
data_inspections = raw_dataset_2[inspections_col_req]

In [5]:
# Clean-up the new dataframes by either splitting columns or merging multiple rows

# Facilities dataset combines several attributes into one column, split the attributes
def facilities_column_split(facilities):
    # split columns
    facilities["CAT_1"], facilities["CAT_2"] = facilities["CATEGORY"].str.split(",", 1).str
    facilities["SUBCAT_1"], facilities["SUBCAT_2"], facilities["SUBCAT_3"] = facilities["SUBCATEGORY"].str.split("/", 2).str

    # drop the old columns
    facilities.drop('CATEGORY', axis=1, inplace=True)
    facilities.drop('SUBCATEGORY', axis=1, inplace=True)
    
    return facilities

def infractions_aggregate_rows(df):
    # add a counter column, (that will be removed later)
    df = df.assign(InfractionCount=1)
    
    # sum all identical rows in a new column
    df['INFRACTION_TOTAL'] = df.groupby(["INSPECTION_ID", "INFRACTION_TYPE", "Result", "InspectionDate"])["InfractionCount"].transform('sum')
    
    # drop all but the first occurance of duplicate rows
    df.drop_duplicates(subset=["INSPECTION_ID", "INFRACTION_TYPE", "Result", "InspectionDate"], keep='first', inplace=True)
    
    # drop temporary InfractionCount column
    df.drop('InfractionCount', axis=1, inplace=True)
    
    return df

# Apply changes

data_facilities = facilities_column_split(data_facilities)
data_infractions = infractions_aggregate_rows(data_infractions)

In [6]:
data_facilities.head()

,FACILITYID,BUSINESS_NAME,ADDR,CITY,CAT_1,CAT_2,SUBCAT_1,SUBCAT_2,SUBCAT_3
0,A1502042-EED8-416B-91D2-02717F07E5C7,"OAKS PRESCHOOL, THE",147 ELGIN ST N,CAMBRIDGE,Food,Institutional,Child Care Onsite Prep.,Recreational Camp,Co-op
1,4DFF46E1-FC0D-4AC4-8824-08791F7FFDF9,YMCA CHILD CARE-WILLIAMSBURG,760 COMMONWEALTH CRES,KITCHENER,Food,Institutional,Child Care Onsite Prep.,Recreational Camp,Co-op
2,BFA8C5FA-9B7B-4C8C-86B1-0E3387FC62F5,PEEKABOO CHILD CARE MAPLE GROVE,1370 MAPLE GROVE RD,CAMBRIDGE,Food,Institutional,Child Care Onsite Prep.,Recreational Camp,Co-op
3,F6DE1A2D-0F8E-4622-8F40-0E85664E81C7,UW - EARLY CHILDHOOD EDUCATIONAL CENTRE,200 UNIVERSITY AVE W,WATERLOO,Food,Institutional,Child Care Onsite Prep.,Recreational Camp,Co-op
4,54EA4088-0FC5-4B59-863E-112BD8987F2B,JUMPING JACKS CHILD CARE CENTRE,685 MYERS RD,CAMBRIDGE,Food,Institutional,Child Care Onsite Prep.,Recreational Camp,Co-op


In [7]:
data_infractions.head()

,INSPECTION_ID,INFRACTION_TYPE,Result,InspectionDate,INFRACTION_TOTAL
0,6E34D7BE-BD3B-4E43-8EC8-7294E036EA21,NON-CRITICAL,Corrected During Inspection,2017-05-13,1
1,B256E701-9EE1-4709-9000-3B89CD5EEB55,NON-CRITICAL,Not in Compliance,2016-05-25,2
3,C6EA66F6-021A-4552-BA79-8B8E349B371F,NON-CRITICAL,Not in Compliance,2017-08-23,3
6,C6EA66F6-021A-4552-BA79-8B8E349B371F,NON-CRITICAL,Corrected During Inspection,2017-08-23,1
7,43ED12C3-3B1C-4F7A-9EE6-93C6DBD5C991,NON-CRITICAL,Not in Compliance,2017-06-28,2
